## Import Libraries and APIs

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [4]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data=requests.get(api_url).json()
data
# api_url

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'NGDS(ST )ALCANEBT LGOKALSEM/RA EQ',
 'calculationPrice': 'iexlasttrade',
 'open': 0,
 'openTime': None,
 'openSource': 'iaffloci',
 'close': 0,
 'closeTime': None,
 'closeSource': 'ofliafci',
 'high': 0,
 'highTime': None,
 'highSource': None,
 'low': 0,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 149.507,
 'latestSource': 'IEX Last Trade',
 'latestTime': 'July 15, 2021',
 'latestUpdate': 1684930279427,
 'latestVolume': None,
 'iexRealtimePrice': 153.935,
 'iexRealtimeSize': 103,
 'iexLastUpdated': 1629828229636,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 149.72,
 'previousVolume': 132856541,
 'change': -0.675,
 'changePercent': -0.00462,
 'volume': None,
 'iexMarketPercent': 0.01798579490003522,
 'iexVolume': 19464

## Parsing API Call

In [5]:
print(data['latestPrice'])
print(data['marketCap'])

149.507
2543902282950


### Add stocks data to a Pandas DataFrame

In [6]:
my_columns = ['Ticker', 'Price', 'Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
# final_dataframe

In [7]:
final_dataframe = final_dataframe.append(pd.Series(['AAPL',
                                                    data['latestPrice'],
                                                    data['marketCap'],
                                                    'N/A'],
                                                    index = my_columns),
                                         ignore_index=True)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,149.507,2543902282950,N/A


In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data =requests.get(api_url).json()
    final_dataframe = final_dataframe.append(pd.Series([symbol,
                                                    data['latestPrice'],
                                                    data['marketCap'],
                                                    'N/A'],
                                                    index = my_columns),
                                         ignore_index=True)

In [9]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,150.100,45291571248,N/A
1,AAL,21.400,13235073017,N/A
2,AAP,219.540,14214179621,N/A
3,AAPL,154.139,2532120676680,N/A
4,ABBV,119.120,216613425645,N/A
...,...,...,...,...
500,YUM,121.460,36076125065,N/A
501,ZBH,156.400,32939371090,N/A
502,ZBRA,544.100,28743272103,N/A
503,ZION,53.900,8747901467,N/A


### Optimization with Batch API Calls

In [10]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [11]:
symbol_groups=list(chunks(stocks['Ticker'],100))
symbol_strings =[]
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe =final_dataframe.append(pd.Series([symbol,
                                                          data[symbol]['quote']['latestPrice'],
                                                          data[symbol]['quote']['marketCap'],
                                                          'N/A'],
                                                          index=my_columns),
                                               ignore_index=True)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,153.220,46454798508,N/A
1,AAL,20.570,13674772041,N/A
2,AAP,217.270,14035549182,N/A
3,AAPL,150.291,2581307793427,N/A
4,ABBV,120.440,208760681033,N/A
...,...,...,...,...
500,YUM,117.260,36108312087,N/A
501,ZBH,160.300,32643266057,N/A
502,ZBRA,527.450,28425442197,N/A
503,ZION,52.600,8686391129,N/A


### Calculate how many number of shares to buy

In [12]:
portfolio_size= input("Enter the value of your portfolio")

try:
    val=float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio100000


In [13]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i,'Number Of Shares to Buy'] = math.floor(position_size/final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,153.220,46454798508,1
1,AAL,20.570,13674772041,9
2,AAP,217.270,14035549182,0
3,AAPL,150.291,2581307793427,1
4,ABBV,120.440,208760681033,1
...,...,...,...,...
500,YUM,117.260,36108312087,1
501,ZBH,160.300,32643266057,1
502,ZBRA,527.450,28425442197,0
503,ZION,52.600,8686391129,3


### Formatting Excel Output

In [14]:
#initializing XlsxWriter Object
writer = pd.ExcelWriter('recommended_trades.xlsx',engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

#### Arguments required to format `.xlsx` file

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [16]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number Of Shares to Buy', integer_format],
}
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',20,column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',column_formats[column][0],string_format)

In [17]:
writer.save()